task 2.1: Make a DataModel class that reads the output of the DataAquisition class (from task1) in its __init__()  

task 2.2: Make a function in DataModel to make a linear regression. I suggest you try to do it on your own with only vanilla python and the class notes. If you are lost, you can find practical info here on how to use already made libraries: https://www.geeksforgeeks.org/machine-learning/regularization-in-machine-learning/Lenker til en ekstern side.  The issue here will be data structure: np.array vs list of list vs pandas DataFrames.  

task 2.3: Make a function that split the data you got from DataAquisition into train, validation and test. Do it  with vanilla python. You need to make sure you understand the data structure.  

task 2.4: Make a function that computes MSE (make your own, don't copy from my notes :P )  

task 2.5: Make a function to make NN. It would be essentially a wrapper of other libraries, I suggest to use Keras:  https://www.geeksforgeeks.org/machine-learning/how-to-create-models-in-keras/  . You should have acquired enough notions to handle this tool.  

task 2.6: Make a function that does K_MEAN and GMM (we will discuss them next week)  

Once these methods (recipes) are done, you can now make cakes! :) :  

task 2.7: Make a linear regression on all your data (statistic ).  

task 2.8: Make a linear regression on all your train data and test it on your validation.  

task 2.9: Compute the MSE on your validation data.   

task 2.10: Try for different distribution of initial data point, (a) Discuss how different functions can be used in the linear regression, and different NN architecture. (b) Discuss how you can use the validation data for the different cases. (c) Discuss the different outcome from the different models when using the full dataset to train and when you use a different ML approach. (d) Discuss the outcomes you get for K-means and GMM. (e) Discuss how you can integrate supervised and unsupervised methods for your case.  

In [ ]:
import sys
sys.path.append('../MOD550-task1') # Adjust the path to find data_aquisition module

from data_aquisition import DataAquisition as aq
import random


class DataModel:
    def __init__(self):
        self.data = aq().data

    def linear_regression(self, x, y):
        """
        Performs simple linear regression using the least squares method.
        Args:
            x (list or array-like): The independent variable values.
            y (list or array-like): The dependent variable values.
        Returns:
            tuple: Intercept (b0) and slope (b1) of the regression line.
        """
        n = len(x)
        x_mean = sum(x) / n
        y_mean = sum(y) / n
        numerator = sum((x[i] - x_mean) * (y[i] - y_mean) for i in range(n))
        denominator = sum((x[i] - x_mean) ** 2 for i in range(n))
        b1 = numerator / denominator
        b0 = y_mean - b1 * x_mean
        return b0, b1

    def predict(self, x_new, b0, b1):
        """
        Predicts the dependent variable value for a new independent variable input using the linear regression coefficients.
        Args:
            x_new: The new value of the independent variable.
            b0: Intercept of the regression line.
            b1: Slope of the regression line.
        Returns:
            Predicted value of the dependent variable.
        """
        return b0 + b1 * x_new
    
    def predict_linear_regression(self, x_new, x=None, y=None):
        """
        Predicts the dependent variable value for a new independent variable input using linear regression.
        If x and y are not provided, uses the data loaded in self.data, namely 'Release year' as x and 'Rating' as y.
        Args:
            x_new: The new value of the independent variable to predict for.
            x: List or array-like of independent variable values.
            y: List or array-like of dependent variable values.
        Returns:
            Predicted value of the dependent variable for x_new.
        """
        if x is None and y is None:
            x = self.data['Release year']
            y = self.data['Rating']
        elif x is None or y is None:
            raise ValueError("Both x and y must be provided if one is provided.")
        b0, b1 = self.linear_regression(x, y)
        return self.predict(x_new, b0, b1)

    def train_val_test_split(self, train_ratio=0.7, val_ratio=0.15, seed=1):
        """
        Splits the dataset into train, validation, and test sets after shuffling.
        Args:
            train_ratio: Proportion of data to use for training.
            val_ratio: Proportion of data to use for validation.
            seed (int, optional): Random seed for reproducibility.
        Returns:
            tuple: (train_data, val_data, test_data) as lists of row dicts.
        """
        if train_ratio + val_ratio > 1:
            raise ValueError("Ratios are over 1")
        if seed is not None:
            random.seed(seed)
        # Convert column dict to list of row dicts
        columns = list(self.data.keys())
        n_rows = len(self.data[columns[0]])
        rows = [ {col: self.data[col][i] for col in columns} for i in range(n_rows) ]
        random.shuffle(rows)
        n = len(rows)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)
        train_rows = rows[:n_train]
        val_rows = rows[n_train:n_train + n_val]
        test_rows = rows[n_train + n_val:]
        return train_rows, val_rows, test_rows

    def mean_squared_error(self, actual, predicted):
        """
        Computes the Mean Squared Error (MSE) between actual and predicted values.
        Args:
            actual: True values.
            predicted: Predicted values.
        Returns:
            The mean squared error.
        """
    # Ensure the lists are of the same length
        if len(actual) != len(predicted):
            raise ValueError("Lists must have the same length.")
        n = len(actual)
        squared_errors = 0
        for i in range(n):
            squared_errors += (actual[i] - predicted[i]) ** 2
        mse = squared_errors / n
        return mse
    
    def test_MSE(self):
        """
        Fits a linear regression model on the training data and evaluates its performance on the test data using Mean Squared Error (MSE).

        Returns:
            tuple: (b0, b1, mse)
                b0: Intercept of the regression line fitted on the training data.
                b1: Slope of the regression line fitted on the training data.
                mse: Mean Squared Error of the predictions on the test data.
        """
        data_train, unused , data_test = self.train_val_test_split(train_ratio=0.7, val_ratio=0, seed=1) #To get 30 percent test data)
        # Extract x (Release year) and y (Rating) from train data
        x_train = [row['Release year'] for row in data_train]
        y_train = [row['Rating'] for row in data_train]

        # Fit linear regression on train data
        b0, b1 = self.linear_regression(x_train, y_train)
        x_test = [row['Release year'] for row in data_test]
        y_test = [row['Rating'] for row in data_test]

        # Predict on test data
        y_pred = [self.predict(x, b0, b1) for x in x_test]

        # Calculate MSE
        mse = self.mean_squared_error(y_test, y_pred)
        return b0, b1, mse


**Part 1**  


In [2]:
dataw = DataModel()


File found: ../title.basics.tsv
File found: ../data/title.ratings.tsv


**Part 2**  


In [3]:
dataw.predict_linear_regression(x_new=2040)



np.float64(7.0657166085933705)

**Part 3**

In [4]:
data_train, data_validation, data_test = dataw.train_val_test_split(seed=1)


In [13]:
print("Length of training set:", len(data_train))
print("Length of validation set:", len(data_validation))
print("Length of test set:", len(data_test))


Length of training set: 191088
Length of validation set: 40947
Length of test set: 40949


**Part 4**

In [ ]:
model_b0, model_b1, model_mse = dataw.test_MSE()
print("Intercept (b0):", model_b0, "Slope (b1):", model_b1, "MSE:", model_mse)

Intercept (b0): -6.3419693651290885 Slope (b1): 0.006570642790351797 MSE: 1.834137841834273


**Part 5**

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

2025-10-08 07:38:59.687220: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-08 07:39:00.581056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 07:39:04.021073: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [16]:
def make_nn(input_dim, layers_units=[64, 32], activation='relu', output_activation='linear', optimizer='adam', loss='mse'):
    """
    Creates and compiles a simple feedforward neural network using Keras.
    
    Args:
        input_dim (int): Number of input features.
        layers_units (list): List with the number of units in each hidden layer.
        activation (str): Activation function for hidden layers.
        output_activation (str): Activation function for output layer.
        optimizer (str): Optimizer to use.
        loss (str): Loss function to use.
        
    Returns:
        keras.Model: Compiled Keras model.
    """
    model = keras.Sequential()
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))
    # Hidden layers
    for units in layers_units:
        model.add(layers.Dense(units, activation=activation))
    # Output layer
    model.add(layers.Dense(1, activation=output_activation))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [18]:
import numpy as np

# We'll use the make_nn function to try different architectures and find the one with the lowest validation MSE.
# We'll try a few combinations of hidden layers and units.

def evaluate_nn_architectures(x_train, y_train, x_val, y_val, architectures, epochs=5, batch_size=128):
    results = []
    for arch in architectures:
        model = make_nn(input_dim=x_train.shape[1], layers_units=arch)
        history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(x_val, y_val))
        val_mse = history.history['val_loss'][-1]
        results.append({'architecture': arch, 'val_mse': val_mse})
    return sorted(results, key=lambda x: x['val_mse'])

# Prepare data: extract features and target from data_train and data_validation

def extract_xy(data):
    # Use 'Release year' as feature, 'Rating' as target
    x = np.array([[row['Release year']] for row in data])
    y = np.array([row['Rating'] for row in data])
    return x, y

x_train, y_train_nn = extract_xy(data_train)
x_val, y_val_nn = extract_xy(data_validation)

# Try different architectures
architectures = [
    [32],
    [64],
    [128],
    [64, 32],
    [128, 64],
    [128, 64, 32]
]

results = evaluate_nn_architectures(x_train, y_train_nn, x_val, y_val_nn, architectures, epochs=3, batch_size=256)
best = results[0]
print("Best architecture:", best['architecture'], "Validation MSE:", best['val_mse'])

Best architecture: [64, 32] Validation MSE: 1.8335663080215454


**Part 7**